**Import Libs**

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

**Load Dataset**

In [0]:
data_frame = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Content-Based Recommender System Data/data-10-fv-tf-idf-scores-7477-row-31-class-200-feature.csv', header=None, sep=',')
# data_frame = data_frame_unshuffled.sample(frac=1).reset_index(drop=True)

In [4]:
header = []

for i in range(len(data_frame.columns)-1):
  header.append(str(i))

header.append('target')
data_frame.columns = header

data_frame.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,5661,5662,5663,5664,5665,5666,5667,5668,5669,5670,5671,5672,5673,5674,5675,5676,5677,5678,5679,5680,5681,5682,5683,5684,5685,5686,5687,5688,5689,5690,5691,5692,5693,5694,5695,5696,5697,5698,5699,target
0,0.613706,0.000000,0.0,0.0,0.613706,0.0,0.0,0.0,0.0,0.306853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.000000,0.306853,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.920558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.306853,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.613706,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.920558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.613706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
x = data_frame.drop('target',axis=1)
y = data_frame['target']

x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(x, y, test_size=0.2)

x_train = x_train_df.to_numpy()
x_test = x_test_df.to_numpy()
y_train = y_train_df.to_numpy()
y_test = y_test_df.to_numpy()

print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(5980, 5700) (1495, 5700)
(5980,) (1495,)


**Hyperparameters**

In [0]:
num_features = 5700
num_labels = 31
learning_rate = 0.001
batch_size = 64
training_epochs = 1000
display_step = 1

**One-Hot Encoding**

In [7]:
def to_onehot(y):
  data = np.zeros((num_labels))
  data[y] = 1
  return data

x_train_one_hot_encoded = np.reshape(x_train, (-1, num_features))
x_test_one_hot_encoded = np.reshape(x_test, (-1, num_features))

y_train_one_hot_encoded = np.array([to_onehot(y) for y in y_train])
y_test_one_hot_encoded = np.array([to_onehot(y) for y in y_test])

print(x_train_one_hot_encoded.shape, x_test_one_hot_encoded.shape)
print(y_train_one_hot_encoded.shape, y_test_one_hot_encoded.shape)

(5980, 5700) (1495, 5700)
(5980, 31) (1495, 31)


**Model**

In [0]:
x = tf.placeholder(tf.float32, shape = [None, num_features])
y = tf.placeholder(tf.float32, shape = [None, num_labels])

weight = tf.Variable(tf.zeros([num_features, num_labels]))
bias = tf.Variable(tf.zeros([num_labels]))

pred = tf.nn.softmax(tf.matmul(x, weight) + bias)
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [0]:
def get_accuracy(predictions, labels):
  correctly_predicted = np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
  accuracy = (100.0 * correctly_predicted) / predictions.shape[0]
  return accuracy

In [0]:
with tf.Session() as sess:
  # Run the initializer
  sess.run(init)
  
  # Training cycle
  for epoch in range(training_epochs):
    avg_cost = 0.0
    avg_acc = 0.0
    total_batch = len(x_train_one_hot_encoded)//batch_size
    
    for i in range(total_batch):
      batch_x = x_train_one_hot_encoded[i:i+1*batch_size]
      batch_y = y_train_one_hot_encoded[i:i+1*batch_size]
            
      _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
      avg_cost += c/total_batch
            
      pred_y = sess.run(pred, feed_dict={x: batch_x})
      acc = get_accuracy(pred_y, batch_y)
      avg_acc += acc/total_batch
            
            
      if (epoch+1) % display_step == 0:
        tc = sess.run(cost, feed_dict={x: x_test_one_hot_encoded, y: y_test_one_hot_encoded})
        pred_y = sess.run(pred, feed_dict={x: x_test_one_hot_encoded})
        ta = get_accuracy(pred_y, y_test_one_hot_encoded)
            
        print("Epoch: {:2.0f} - Cost: {:1.5f} - Acc: {:0.5f} - Test Cost: {:0.5f} - Test Acc: {:0.5f}".format(
            epoch+1, avg_cost, avg_acc, tc, ta))
            
  print("Optimization Finshed")

  # Test model
  correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
  # Calculate accuracy
  acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print("Test Accuracy:", acc.eval({x: x_test_one_hot_encoded, y: y_test_one_hot_encoded}))

Streaming output truncated to the last 5000 lines.
Epoch: 749 - Cost: 0.04768 - Acc: 3.52823 - Test Cost: 3.22628 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.05897 - Acc: 4.45228 - Test Cost: 3.22626 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.06993 - Acc: 5.39315 - Test Cost: 3.22624 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.08066 - Acc: 6.35081 - Test Cost: 3.22623 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.09120 - Acc: 7.30847 - Test Cost: 3.22621 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.10166 - Acc: 8.26613 - Test Cost: 3.22619 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.11185 - Acc: 9.22379 - Test Cost: 3.22617 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.12183 - Acc: 10.18145 - Test Cost: 3.22616 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.13145 - Acc: 11.15591 - Test Cost: 3.22615 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.14080 - Acc: 12.14718 - Test Cost: 3.22613 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.15002 - Acc: 13.13844 - Test Cost: 3.22613 - Test Acc: 16.05351
Epoch: 749 - Cost: 0.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Epoch: 802 - Cost: 0.48229 - Acc: 76.29368 - Test Cost: 3.23547 - Test Acc: 15.91973
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-31436bc8a85a>", line 24, in <module>
    tc = sess.run(cost, feed_dict={x: x_test_one_hot_encoded, y: y_test_one_hot_encoded})
  File "/tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py", line 956, in run
    run_metadata_ptr)
  File "/tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py", line 1149, in _run
    np_val = np.asarray(subfeed_val, dtype=subfeed_dtype)
  File "/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py", line 85, in asarray
    return array(a, dtype, copy=False, order=order)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/py

KeyboardInterrupt: ignored

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 438, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 657, in _abort_queue
    sel